In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price_4bedroom
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
""").show()

+----+------------------+
|year|avg_price_4bedroom|
+----+------------------+
|2019|          300263.7|
|2020|         298353.78|
|2021|         301819.44|
|2022|         296363.88|
+----+------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3bedroom
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""").show()

+----------+------------------+
|date_built|avg_price_3bedroom|
+----------+------------------+
|      2010|         292859.62|
|      2011|         291117.47|
|      2012|         293683.19|
|      2013|         295962.27|
|      2014|         290852.27|
|      2015|          288770.3|
|      2016|         290555.07|
|      2017|         292676.79|
+----------+------------------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3by3
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""").show()

+----------+--------------+
|date_built|avg_price_3by3|
+----------+--------------+
|      2010|     285010.22|
|      2011|     276553.81|
|      2012|     307539.97|
|      2013|     303676.79|
|      2014|     298264.72|
|      2015|     297609.97|
|      2016|      293965.1|
|      2017|     280317.58|
+----------+--------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
|  51|     788128.21|
|  52|     733780.26|
|  53|      755214.8|
|  54|     798684.82|
|  55|     771153.32|
|  56|      718176.4|
|  57|      734340.5|
|  58|     759764.65|
|  59|      791453.0|
|  60|     754939.65|
|  61|     746877.59|
|  62|     759150.14|
|  63|     711614.55|
|  64|     767036.67|
|  65|     736679.93|
|  66|      712475.0|
|  67|     737970.96|
|  68|     716785.44|
|  69|     750537.94|
|  70|     695865.58|
+----+--------------+
only showing top 20 rows

--- 0.8189561367034912 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

cached_query = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
cached_query.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------+
|view|avg_price_view|
+----+--------------+
|  51|     788128.21|
|  52|     733780.26|
|  53|      755214.8|
|  54|     798684.82|
|  55|     771153.32|
|  56|      718176.4|
|  57|      734340.5|
|  58|     759764.65|
|  59|      791453.0|
|  60|     754939.65|
|  61|     746877.59|
|  62|     759150.14|
|  63|     711614.55|
|  64|     767036.67|
|  65|     736679.93|
|  66|      712475.0|
|  67|     737970.96|
|  68|     716785.44|
|  69|     750537.94|
|  70|     695865.58|
+----+--------------+
only showing top 20 rows

--- 0.4233083724975586 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "home_sales_partitioned"
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)

In [ ]:
# 11. Read the parquet formatted data.
parquet_path = "home_sales_partitioned"
df_parquet = spark.read.parquet(parquet_path)
df_parquet.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

result_parquet = spark.sql(query)

result_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.7952761650085449 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")